# Analysis of BioTek plate reader growth curves

## Date:

(c) 2020 Manuel Razo. This work is licensed under a [Creative Commons Attribution License CC-BY 4.0](https://creativecommons.org/licenses/by/4.0/). All code contained herein is licensed under an [MIT license](https://opensource.org/licenses/MIT)

---

In [ ]:
import os
import itertools
import git

# Our numerical workhorses
import numpy as np
import scipy as sp
import scipy.signal
import pandas as pd

# Import matplotlib stuff for plotting
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib as mpl

# Seaborn, useful for graphics
import seaborn as sns

# Import Interactive plot libraries
import bokeh.plotting
import bokeh.layouts
from bokeh.themes import Theme
import holoviews as hv
import hvplot
import hvplot.pandas
import panel as pn

# Import the project utils
import fit_seq 

# This enables SVG graphics inline
%config InlineBackend.figure_format = 'retina'

bokeh.io.output_notebook()
hv.extension('bokeh')

In [ ]:
# Set PBoC plotting format
evo_mwc.viz.pboc_style_mpl()
# Increase dpi
mpl.rcParams['figure.dpi'] = 110

# Set PBoC style for plot
theme = Theme(json=evo_mwc.viz.pboc_style_bokeh())
hv.renderer('bokeh').theme = theme

## Purpose
Description of the experiment.

  
 ## Strains

| Plasmid | Genotype | Host Strain | Shorthand |
| :------ | :------- | ----------: | --------: |
| `pZS-eample`| `galK<>example` |  HG105 |`R0` |

### Run metadata

In [ ]:
# Find home directory for repo
repo = git.Repo("./", search_parent_directories=True)
homedir = repo.working_dir

# Find date
workdir = os.getcwd().split('/')[-1]
DATE = int(workdir.split('_')[0])
RUN_NO = int(workdir.split('_')[1][-1])

## Per-well growth rate analysis

Let's begin by importing the growth rates as inferred with the Gaussian process method. We will start with the per-well analysis.

In [ ]:
df_gp = pd.read_csv(
    f"./output/{DATE}_r{RUN_NO}_gp_per_well.csv", index_col=False
)
# Specify row and columns
df_gp["row"] = [x[0] for x in df_gp.well]
df_gp["col"] = [x[1::] for x in df_gp.well]


def sort_by_time(df, time="time_min"):
    """
    Function to sort each well data by time
    """
    return df.sort_values(by=time)


# Apply function and then apply drop level to remove resulting multiindex
df_gp = df_gp.groupby("well").apply(sort_by_time).droplevel(level=0)

df_gp.head()

### Whole-plate growth curves

Let's quickly take a look at all raw data from each well. This is just a rough look at the kind of data we are looking at.

In [ ]:
hv.output(size=50)
# Generate hvplot
df_gp.hvplot(
    x="time_min",
    y="OD600",
    row="col",
    col="row",
    xlabel="time (min)",
    ylabel="OD600",
    xticks=3,
    yticks=3,
)

### Whole-plate growth rates

Now let's take a look at all of the growth rates.

In [ ]:
hv.output(size=50)
# Generate hvplot
df_gp.hvplot(
    x="time_min",
    y="growth_rate",
    row="col",
    col="row",
    xlabel="time (min)",
    ylabel="λ (min\u207B\u00B9)",
    xticks=3,
    yticks=3,
)

### Per-well growth curves (with `HoloViews`)

These measurements are really noisy, especially at the beginning of the growth curves. Let's take a look at the individual trajectories. For this we will use `HoloViews` rather than `hvplot` to quickly get an interactive widget with which change the curve.

In [ ]:
# Generate curves per well with dropdown menu
hv_OD = hv.Curve(
    data=df_gp,
    kdims=[("time_min", "time (min)",), ("OD600", "OD600")],
    vdims=["well"],
).groupby("well")

hv_gr = hv.Curve(
    data=df_gp,
    kdims=[
        ("time_min", "time (min)",),
        ("growth_rate", "growth rate (min\u207B\u00B9)"),
    ],
    vdims=["well"],
).groupby("well")

# Generate layout for plots on top of each other
hv_layout = hv.Layout(
    hv_OD.opts(width=800, height=400, xlabel="")
    + hv_gr.opts(width=800, height=400)
).cols(1)
hv_layout

### grouped curves (with `Panel`)

Another way to display these data is by grouping by the strain and the selection they grew in rather than by single well to account for technical replicates.

In [ ]:
# Define widgets to interact with plot
# unique strains
strain = pn.widgets.CheckButtonGroup(
    name="strain", options=list(df_gp.strain.unique()), button_type="primary"
)
# negative selection
pos_selection = pn.widgets.CheckButtonGroup(
    name="pos selection",
    options=list(df_gp.pos_selection.unique()),
    button_type="success",
)
# time window
time_slider = pn.widgets.RangeSlider(
    name="time range (min)",
    start=0,
    end=df_gp.time_min.max(),
    value=(0, df_gp.time_min.max()),
    step=5,
)

# Generate function to plot the data
@pn.depends(
    strain.param.value, pos_selection.param.value, time_slider.param.value
)
def plot_groups(strain, pos_selection, time_slider):
    """
    Function to generate interactive plot
    """
    # Initialize list to save plots
    od_plots = list()
    gr_plots = list()
    # Generate all pairs of groups
    groups = list(itertools.product(strain, pos_selection))
    
    # Define colors for groups
    colors = bokeh.palettes.Category10[10][0:len(groups)]
    # Set colors in dictionary
    col_dict = dict(zip(groups, colors))
    # Check if groups are being selected
    if len(groups) > 0:
        # Initialize figure for OD600
        od_fig = bokeh.plotting.figure(
            width=800,
            height=250,
            x_axis_label="",
            y_axis_label="OD600",
            toolbar_location="above",
            tools=["box_zoom", "pan", "wheel_zoom", "reset"],
        )
        # Add legend outside plot
        od_fig.add_layout(bokeh.models.Legend(), 'right')
        
        # Initialize figure for growth rate
        gr_fig = bokeh.plotting.figure(
            width=800,
            height=250,
            x_axis_label="time (min)",
            y_axis_label="growth rate",
            toolbar_location=None,
            x_range=od_fig.x_range,
            tools=""
        )
        
        # Loop through groups and plot each group
        for group in groups:
            # Extract data
            data = df_gp[
                (df_gp["strain"] == group[0])
                & (df_gp["pos_selection"] == group[1])
            ].sort_values(by="time_min")
            
            # Group by wells
            data_group = data.groupby("well")
            
            # Loop through wells
            for i, (g, df) in enumerate(data_group):
                # Declare bokeh data source
                source = bokeh.models.ColumnDataSource(df)
                
                # Plot growth curve
                od_fig.line(
                    x="time_min",
                    y="OD600",
                    source=source,
                    line_width=1.5,
                    legend_label=f"{group[0]} - {group[1]}",
                    color=col_dict[group],
                    
                )
                # Plot growth curve
                gr_fig.line(
                    x="time_min",
                    y="growth_rate",
                    source=source,
                    line_width=1.5,
                    color=col_dict[group],
                )
            # Add hover tools to check which well
            od_fig.add_tools(bokeh.models.HoverTool(tooltips=[("well", "@well")]))
            gr_fig.add_tools(bokeh.models.HoverTool(tooltips=[("well", "@well")]))
                
        # Apply PBoC format
        evo_mwc.viz.pboc_single(od_fig)
        evo_mwc.viz.pboc_single(gr_fig)
        
        return pn.pane.Bokeh(bokeh.layouts.column([od_fig, gr_fig]))

pn.Column(
    strain,
    pos_selection,
    time_slider,
    plot_groups,
)